In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

# Load historical dataset
historical_data = pd.read_csv('/content/cleaned_data.csv')

# Extract metadata (station details)
meta_columns = ['station_id', 'station_name', 'region_name', 'latitude', 'longitude',
                'region_id', 'province_id', 'province_name']
meta_data = historical_data[meta_columns].drop_duplicates()

# Group by station_id and month to get mean and std for each feature (historical behavior)
# Convert 'Date' column to datetime, specifying the correct format
historical_data['Date'] = pd.to_datetime(historical_data['Date'], dayfirst=True)

# Extract the month
historical_data['Month'] = historical_data['Date'].dt.month
weather_columns = [
    'Min Temperature', 'Max Temperature', 'Average Temperature', 'Average Humidity',
    'Rainfall', 'Sunshine Duration', 'Max Wind Speed', 'Wind Direction at Max Speed',
    'Average Wind Speed', 'Most Wind Direction'
]

# Create dictionary to store historical mean and std for each station and month
station_month_stats = {}

for station in meta_data['station_id'].unique():
    station_data = historical_data[historical_data['station_id'] == station]

    # Group by station and month to calculate mean and std deviation for numeric columns
    station_month_stats[station] = station_data.groupby('Month')[weather_columns[:-1]].agg(['mean', 'std']).to_dict()

    # For non-numeric columns like Wind Direction, calculate the most frequent (mode) value
    wind_direction_mode = station_data.groupby('Month')['Wind Direction at Max Speed'].agg(lambda x: x.mode()[0]).to_dict()
    station_month_stats[station]['Wind Direction at Max Speed'] = wind_direction_mode

# Define date range for synthetic data (2021–2024)
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date)

# List of possible categories for 'Most Wind Direction'
wind_direction_categories = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'C']

# Define valid ranges for each column
valid_ranges = {
    'Min Temperature': (5, 28),
    'Max Temperature': (25, 40),
    'Average Temperature': (20, 38),
    'Average Humidity': (60, 100),
    'Rainfall': (0, 500),
    'Sunshine Duration': (0, 12),
    'Max Wind Speed': (0, 30),
    'Average Wind Speed': (0, 30),
    'Wind Direction at Max Speed': (0, 360)
}

# Create a list to store synthetic data
synthetic_data = []

# Function to generate synthetic angular data using circular normal distribution
def generate_circular_normal(mu, sigma):
    angle = np.random.normal(mu, sigma)
    return angle % 360  # Ensure between 0 and 360 degrees

# Function to filter values based on valid ranges
def apply_valid_range(value, valid_range):
    min_val, max_val = valid_range
    return min_val <= value <= max_val if not np.isnan(value) else True

# Generate synthetic data for each day, station, and season
for station in meta_data['station_id'].unique():
    station_meta = meta_data[meta_data['station_id'] == station].iloc[0]
    station_stats = station_month_stats[station]

    for single_date in date_range:
        month = single_date.month
        year = single_date.year

        # Get monthly stats (mean and std) for the current station and month
        month_stats = {col: (
            station_stats.get((col, 'mean'), {}).get(month, 0),  # Replace None with 0 if missing
            station_stats.get((col, 'std'), {}).get(month, 1)    # Replace None with 1 if missing
        ) for col in weather_columns[:-1]}

        # Apply seasonal adjustments (e.g., temperature in summer months may be higher)
        if month in [6, 7, 8]:  # Summer months (adjust temperature upward)
            temperature_adjustment = np.random.uniform(1, 3)  # Adding slight variation for hotter months
        elif month in [12, 1, 2]:  # Winter months (adjust temperature downward)
            temperature_adjustment = np.random.uniform(-2, -1)  # Adding slight variation for colder months
        else:
            temperature_adjustment = 0

        # Generate synthetic weather data using normal distribution
        min_temp = np.random.normal(month_stats['Min Temperature'][0] + temperature_adjustment, month_stats['Min Temperature'][1])
        max_temp = np.random.normal(month_stats['Max Temperature'][0] + temperature_adjustment, month_stats['Max Temperature'][1])
        avg_temp = np.random.normal(month_stats['Average Temperature'][0] + temperature_adjustment, month_stats['Average Temperature'][1])
        avg_humidity = np.random.normal(month_stats['Average Humidity'][0], month_stats['Average Humidity'][1])
        rainfall = np.random.normal(month_stats['Rainfall'][0], month_stats['Rainfall'][1])
        sunshine_duration = np.random.normal(month_stats['Sunshine Duration'][0], month_stats['Sunshine Duration'][1])
        max_wind_speed = np.random.normal(month_stats['Max Wind Speed'][0], month_stats['Max Wind Speed'][1])
        avg_wind_speed = np.random.normal(month_stats['Average Wind Speed'][0], month_stats['Average Wind Speed'][1])

        # Apply valid ranges for all columns (except 'Wind Direction at Max Speed' and 'Most Wind Direction')
        min_temp = min_temp if apply_valid_range(min_temp, valid_ranges['Min Temperature']) else np.nan
        max_temp = max_temp if apply_valid_range(max_temp, valid_ranges['Max Temperature']) else np.nan
        avg_temp = avg_temp if apply_valid_range(avg_temp, valid_ranges['Average Temperature']) else np.nan
        avg_humidity = avg_humidity if apply_valid_range(avg_humidity, valid_ranges['Average Humidity']) else np.nan
        rainfall = rainfall if apply_valid_range(rainfall, valid_ranges['Rainfall']) else np.nan
        sunshine_duration = sunshine_duration if apply_valid_range(sunshine_duration, valid_ranges['Sunshine Duration']) else np.nan
        max_wind_speed = max_wind_speed if apply_valid_range(max_wind_speed, valid_ranges['Max Wind Speed']) else np.nan
        avg_wind_speed = avg_wind_speed if apply_valid_range(avg_wind_speed, valid_ranges['Average Wind Speed']) else np.nan

        # Wind Direction at Max Speed: Circular normal distribution, then round to integer
        wind_direction_mu = month_stats.get('Wind Direction at Max Speed', (180, 30))[0]
        wind_direction_sigma = month_stats.get('Wind Direction at Max Speed', (180, 30))[1]
        # Handle NaNs by assigning default values if needed
        if pd.isna(wind_direction_mu):
          wind_direction_mu = 180  # default mean
        if pd.isna(wind_direction_sigma) or wind_direction_sigma == 0:
          wind_direction_sigma = 30  # default std deviation

        wind_direction = round(generate_circular_normal(wind_direction_mu, wind_direction_sigma))

        # Most Wind Direction: Randomly select from categories
        most_wind_direction = np.random.choice(wind_direction_categories)

        # Create a row for the synthetic data
        row_data = {
            'Date': single_date.strftime('%d-%m-%Y'),
            'Min Temperature': min_temp,
            'Max Temperature': max_temp,
            'Average Temperature': avg_temp,
            'Average Humidity': avg_humidity,
            'Rainfall': rainfall,
            'Sunshine Duration': sunshine_duration,
            'Max Wind Speed': max_wind_speed,
            'Wind Direction at Max Speed': wind_direction,
            'Average Wind Speed': avg_wind_speed,
            'Most Wind Direction': most_wind_direction,
            'station_id': station,
            'station_name': station_meta['station_name'],
            'region_name': station_meta['region_name'],
            'latitude': station_meta['latitude'],
            'longitude': station_meta['longitude'],
            'region_id': station_meta['region_id'],
            'province_id': station_meta['province_id'],
            'province_name': station_meta['province_name'],
            'Month-Year': f"{single_date.month}-{single_date.year}"
        }

        synthetic_data.append(row_data)

# Convert generated data into a DataFrame
synthetic_df = pd.DataFrame(synthetic_data)

# Fill missing values using forward fill (ffill) or backward fill (bfill)
synthetic_df.fillna(method='ffill', inplace=True)  # Use forward fill to fill null values
# OR
# synthetic_df.fillna(method='bfill', inplace=True)  # Use backward fill to fill null values


print("Synthetic weather data for 2021-2024 generated and missing values filled successfully!")


<ipython-input-24-98974c7498a1>:159: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  synthetic_df.fillna(method='ffill', inplace=True)  # Use forward fill to fill null values


Synthetic weather data for 2021-2024 generated and missing values filled successfully!


In [ ]:
synthetic_df.head()

,Date,Min Temperature,Max Temperature,Average Temperature,Average Humidity,Rainfall,Sunshine Duration,Max Wind Speed,Wind Direction at Max Speed,Average Wind Speed,Most Wind Direction,station_id,station_name,region_name,latitude,longitude,region_id,province_id,province_name,Month-Year
0,01-01-2021,21.454288,28.329341,23.962563,88.154467,NaN,11.391638,5.445199,57,3.190428,C,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,1-2021
1,02-01-2021,19.641923,29.324679,25.594465,93.591205,7.011535,11.391638,7.630060,203,4.167000,W,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,1-2021
2,03-01-2021,23.342541,28.773507,25.759538,83.355560,26.667755,0.749017,7.608747,15,2.967055,S,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,1-2021
3,04-01-2021,21.379943,29.133054,24.924783,92.403485,26.667755,0.749017,4.107588,107,3.346239,S,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,1-2021
4,05-01-2021,20.950508,29.986494,24.428280,95.403130,17.584587,8.733956,7.104608,126,2.579948,NW,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,1-2021


In [ ]:
historical_data.head()

,Date,Min Temperature,Max Temperature,Average Temperature,Average Humidity,Rainfall,Sunshine Duration,Max Wind Speed,Wind Direction at Max Speed,Average Wind Speed,...,station_id,station_name,region_name,latitude,longitude,region_id,province_id,province_name,Month-Year,Month
0,2010-01-01,21.4,30.2,27.1,82.0,9.0,0.5,7.0,90,5.0,...,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,Jan-10,1
1,2010-01-02,21.0,29.6,25.7,95.0,24.0,0.2,6.0,90,4.0,...,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,Jan-10,1
2,2010-01-03,20.2,26.8,24.5,98.0,63.0,0.0,5.0,90,4.0,...,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,Jan-10,1
3,2010-01-04,21.0,29.2,25.8,90.0,0.0,0.1,4.0,225,3.0,...,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,Jan-10,1
4,2010-01-08,21.8,29.8,26.8,91.0,3.0,0.6,5.0,90,4.0,...,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam,Jan-10,1


In [ ]:
historical_data = historical_data.drop(columns=['Month'])

In [ ]:
# Combine them row-wise (i.e., stacking one on top of the other)
df = pd.concat([historical_data, synthetic_df], ignore_index=True)


print("Datasets merged successfully!")

Datasets merged successfully!


In [ ]:
df.tail()

,Date,Min Temperature,Max Temperature,Average Temperature,Average Humidity,Rainfall,Sunshine Duration,Max Wind Speed,Wind Direction at Max Speed,Average Wind Speed,Most Wind Direction,station_id,station_name,region_name,latitude,longitude,region_id,province_id,province_name,Month-Year
755502,27-12-2024,21.937092,30.750206,27.087629,84.529674,2.876990,3.915398,2.261745,204,2.821888,W,97980,Stasiun Meteorologi Mopah,Kab. Merauke,-8.52019,140.41568,458,32,Papua,12-2024
755503,28-12-2024,23.088939,32.540324,25.631709,88.917107,0.118304,2.011845,5.718745,232,1.588514,E,97980,Stasiun Meteorologi Mopah,Kab. Merauke,-8.52019,140.41568,458,32,Papua,12-2024
755504,29-12-2024,22.519283,30.821149,24.367143,77.421044,0.118304,9.109493,3.827898,164,3.300838,S,97980,Stasiun Meteorologi Mopah,Kab. Merauke,-8.52019,140.41568,458,32,Papua,12-2024
755505,30-12-2024,23.148396,32.656383,27.555567,76.305661,0.118304,10.119648,2.039664,302,3.062200,NW,97980,Stasiun Meteorologi Mopah,Kab. Merauke,-8.52019,140.41568,458,32,Papua,12-2024
755506,31-12-2024,23.420327,30.232168,25.698893,82.181491,12.307727,0.949633,5.424786,248,3.656509,E,97980,Stasiun Meteorologi Mopah,Kab. Merauke,-8.52019,140.41568,458,32,Papua,12-2024


In [ ]:
# Define valid ranges for each column
valid_ranges = {
    'Min Temperature': (5, 28),
    'Max Temperature': (25, 40),
    'Average Temperature': (20, 38),
    'Average Humidity': (60, 100),
    'Rainfall': (0, 500),
    'Sunshine Duration': (0, 12),
    'Max Wind Speed': (0, 30),
    'Average Wind Speed': (0, 30),
    'Wind Direction at Max Speed': (0, 360)
}

# Apply filtering while keeping NaN values
for col, (min_val, max_val) in valid_ranges.items():
    df = df[(df[col].between(min_val, max_val)) | df[col].isna()]


In [ ]:
# Remove rows where Min Temperature > Max Temperature
condition_min_gt_max_temp = df['Min Temperature'] > df['Max Temperature']
df = df[~condition_min_gt_max_temp]  # Remove these rows
rows_min_gt_max_temp = condition_min_gt_max_temp.sum()
print(f"Rows where Min Temperature > Max Temperature removed: {rows_min_gt_max_temp}")

# Remove rows where Average Temperature is not between Min and Max Temperature
condition_avg_temp_between_min_max = (df['Average Temperature'] <= df['Min Temperature']) | (df['Average Temperature'] >= df['Max Temperature'])
df = df[~condition_avg_temp_between_min_max]  # Remove these rows
rows_avg_temp_between_min_max = condition_avg_temp_between_min_max.sum()
print(f"Rows where Average Temperature is between Min and Max Temperature removed: {rows_avg_temp_between_min_max}")

# Remove rows where Average Wind Speed is greater than Max Wind Speed
condition_avg_wind_speed_less_than_max = df['Average Wind Speed'] > df['Max Wind Speed']
df = df[~condition_avg_wind_speed_less_than_max]  # Remove these rows
rows_avg_wind_speed_less_than_max = condition_avg_wind_speed_less_than_max.sum()
print(f"Rows where Average Wind Speed is less than Max Wind Speed removed: {rows_avg_wind_speed_less_than_max}")

# After removal, count the remaining number of rows
remaining_row_count = df.shape[0]
print(f"Remaining number of rows after removal: {remaining_row_count}")


Rows where Min Temperature > Max Temperature removed: 8
Rows where Average Temperature is between Min and Max Temperature removed: 3380
Rows where Average Wind Speed is less than Max Wind Speed removed: 20662
Remaining number of rows after removal: 731457


In [ ]:
# Count the total number of rows in the dataframe
row_count = df.shape[0]
print(f"Total number of rows: {row_count}")

Total number of rows: 731457


In [ ]:
# Count the number of rows with at least one null value
rows_with_nulls = df.isnull().any(axis=1).sum()

# Display the number of rows with null values
print(f'Number of rows with at least one null value: {rows_with_nulls}')

Number of rows with at least one null value: 1


In [ ]:
# List of columns to fill missing values
numeric_cols = ['Min Temperature', 'Max Temperature', 'Average Temperature',
                'Average Humidity', 'Rainfall', 'Sunshine Duration',
                'Max Wind Speed', 'Average Wind Speed']

def fill_missing_values(df, numeric_cols):
    # Convert 'Date' to datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
    df['Month-Year'] = df['Date'].dt.strftime('%m-%Y')

    # Ensure all numeric columns are in the correct format
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Fill missing values using the mean per region per month-year
    df[numeric_cols] = df.groupby(['region_name', 'Month-Year'])[numeric_cols].transform(lambda x: x.fillna(x.mean()))

    return df

# Apply function
df = fill_missing_values(df, numeric_cols)

# Check remaining null values
print("Remaining Null Values:\n", df.isnull().sum())


Remaining Null Values:
 Date                           0
Min Temperature                0
Max Temperature                0
Average Temperature            0
Average Humidity               0
Rainfall                       0
Sunshine Duration              0
Max Wind Speed                 0
Wind Direction at Max Speed    0
Average Wind Speed             0
Most Wind Direction            0
station_id                     0
station_name                   0
region_name                    0
latitude                       0
longitude                      0
region_id                      0
province_id                    0
province_name                  0
Month-Year                     0
dtype: int64


In [ ]:
# Count the number of rows with at least one null value
rows_with_nulls = df.isnull().any(axis=1).sum()

# Display the number of rows with null values
print(f'Number of rows with at least one null value: {rows_with_nulls}')

Number of rows with at least one null value: 0


In [ ]:
df.to_csv('data_until_2024.csv', index=False)
